[View in Colaboratory](https://colab.research.google.com/github/truongkhanhduy95/Zalo-Landmark-Identification/blob/master/resnet50_train.ipynb)

In [0]:
!wget -NS --content-disposition "https://console.clouderizer.com/givemeinitsh/KEY" && bash ./clouderizer_init.sh



--2018-08-13 15:09:36--  https://console.clouderizer.com/givemeinitsh/UvXZVUuo
Resolving console.clouderizer.com (console.clouderizer.com)... 34.228.182.208
Connecting to console.clouderizer.com (console.clouderizer.com)|34.228.182.208|:443... connected.
HTTP request sent, awaiting response... 
  HTTP/1.1 200 OK
  Server: nginx/1.10.3 (Ubuntu)
  Date: Mon, 13 Aug 2018 15:09:36 GMT
  Content-Type: application/octet-stream; charset=utf-8
  Content-Length: 4471
  Connection: keep-alive
  X-Powered-By: Sails <sailsjs.org>
  Access-Control-Allow-Origin: 
  Access-Control-Allow-Credentials: 
  Access-Control-Allow-Methods: 
  Access-Control-Allow-Headers: 
  Access-Control-Expose-Headers: 
  Content-Disposition: attachment; filename="clouderizer_init.sh"
  ETag: W/"1177-qtpSyNypjgSzTY5dNiKaAw"
  set-cookie: sails.sid=s%3AUIXS5LR4IxO8SUcO-yKbdFxrBw5QWaXF.avS922JIpCPI1akODq1WMdEc3wzmJUKOKhIiUuDz3k8; Path=/; HttpOnly
Length: 4471 (4.4K) [application/octet-stream]
Last-modified header missing --

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

print("TensorFlow version: "+ tf.__version__)

In [0]:
import math, json, os, sys

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing import image

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
print('Files in Drive:')
!ls drive/

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!ls data

In [0]:
!gsutil -m cp -r gs://zalo_challange_landmark/TrainVal/ data
 

In [0]:
!gsutil -m cp -r gs://zalo_challange_landmark/train_val2018.json data


Copying gs://zalo_challange_landmark/train_val2018.json...
- [1/1 files][  2.9 MiB/  2.9 MiB] 100% Done                                    
Operation completed over 1 objects/2.9 MiB.                                      


In [0]:
!mkdir src
!gsutil -m cp -r gs://zalo_challange_landmark/src/ ./

Copying gs://zalo_challange_landmark/src/__pycache__/zalo_utils.cpython-36.pyc...
Copying gs://zalo_challange_landmark/src/inference.py...
Copying gs://zalo_challange_landmark/src/train.py...
Copying gs://zalo_challange_landmark/src/zalo_utils.py...
- [4/4 files][ 25.4 KiB/ 25.4 KiB] 100% Done                                    
Operation completed over 4 objects/25.4 KiB.                                     


In [0]:
!ls


data  datalab  drive  src  TrainVal


In [0]:
!pip3 uninstall torch --yes 
!pip3 uninstall torchvision --yes 
!pip list

Skipping torch as it is not installed.
Uninstalling torchvision-0.2.1:
  Successfully uninstalled torchvision-0.2.1
Package                  Version  
------------------------ ---------
absl-py                  0.3.0    
altair                   2.1.0    
astor                    0.7.1    
beautifulsoup4           4.6.1    
bleach                   2.1.3    
cachetools               2.1.0    
certifi                  2018.4.16
chardet                  3.0.4    
crcmod                   1.7      
cycler                   0.10.0   
decorator                4.3.0    
entrypoints              0.2.3    
future                   0.16.0   
gast                     0.2.0    
google-api-core          1.3.0    
google-api-python-client 1.6.7    
google-auth              1.4.2    
google-auth-httplib2     0.0.3    
google-auth-oauthlib     0.2.0    
google-cloud-bigquery    1.1.0    
google-cloud-core        0.28.1   
google-cloud-language    1.0.2    
google-cloud-storage     1.8.0    
google-cl

In [0]:
import torch
print(torch.__version__)

0.3.0.post4


In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5c954000 @  0x7f723fcde1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.3.0.post4
True


In [0]:
# install PyTorch 
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision


    100% |████████████████████████████████| 592.3MB 49.0MB/s 
    100% |████████████████████████████████| 61kB 5.2MB/s 
    100% |████████████████████████████████| 2.0MB 13.8MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
